In [1]:
import torch
major_version, minor_version = torch.cuda.get_device_capability()
if major_version >= 8:
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    !pip install --no-deps xformers trl peft accelerate bitsandbytes

!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.5/540.5 kB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 20.6 MB/s eta 0:00:00
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-q750ngw0/unsloth_152856c8d86143de9751429ddf3e2baa
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-q750ngw0/unsloth_152856c8d86143de9751429ddf3e2baa
  Resolved https://github.com/unslothai/unsloth.git to commit ecd584a9167e1637b0a0e916af3c5b88690e24fb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 126.8 MB/s eta 0:00:

In [3]:
from unsloth import FastLanguageModel, PatchDPOTrainer
from trl import DPOConfig, DPOTrainer
from datasets import load_dataset # Added to load local files
import torch

# A. SETUP & MODEL LOADING
PatchDPOTrainer()
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3.1-8b-instruct-bnb-4bit",
    max_seq_length = 2048,
    load_in_4bit = True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 32,
    lora_dropout = 0,
    use_gradient_checkpointing = "unsloth",
)

# B. LOAD DATA FROM JSON
# Assume your file is named 'my_dpo_data.json'
# If it's a list of dicts: [{"question": "...", "good": "...", "bad": "..."}, ...]
raw_dataset = load_dataset("json", data_files="/content/DPO_CoT.json", split="train")

def format_dpo_columns(example):
    return {
        "prompt": example["prompt"], # Change "question" to your JSON's prompt key
        "chosen": example["chosen"],   # Change "chosen" to your JSON's preferred key
        "rejected": example["rejected"]# Change "rejected" to your JSON's bad key
    }

dataset = raw_dataset.map(format_dpo_columns)

# C. DPO CONFIGURATION
dpo_config = DPOConfig(
    output_dir="./sql_dpo_results",
    beta=0.1,
    max_length=1024,
    max_prompt_length=512,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=5e-6,
    optim="paged_adamw_32bit",
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=1,
)

# D. INITIALIZE TRAINER
trainer = DPOTrainer(
    model=model,
    ref_model=None,
    args=dpo_config,
    train_dataset=dataset,
    tokenizer=tokenizer,
)

# E. TRAIN
trainer.train()

==((====))==  Unsloth 2026.2.1: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Map:   0%|          | 0/28 [00:00<?, ? examples/s]

Extracting prompt in train dataset (num_proc=6):   0%|          | 0/28 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=6):   0%|          | 0/28 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=6):   0%|          | 0/28 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 28 | Num Epochs = 3 | Total steps = 21
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


TrainOutput(global_step=21, training_loss=0.5248052591369265, metrics={'train_runtime': 272.4216, 'train_samples_per_second': 0.308, 'train_steps_per_second': 0.077, 'total_flos': 0.0, 'train_loss': 0.5248052591369265, 'epoch': 3.0})

In [4]:
import json
import pandas as pd
from unsloth import FastLanguageModel
from tqdm import tqdm # For a nice progress bar

# A. Load model for inference
FastLanguageModel.for_inference(model)

# B. Load your JSON file
with open('DPO_CoT.json', 'r') as f:
    data = json.load(f)

results = []

print(f"Starting inference for {len(data)} questions...")

# C. Loop through all entries
for entry in tqdm(data):
    question = entry["question"]

    # Format the prompt
    messages = [
        {"role": "system", "content": "You are a SQL expert. Output ONLY SQL code."},
        {"role": "user", "content": question}
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda")

    # Generate output
    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=200,
        use_cache=True,
        temperature=0.1,
        do_sample=True
    )

    # Decode result
    prediction = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)

    # Store for comparison
    results.append({
        "Question": question,
        "Chosen_SQL": entry["chosen"],
        "Predicted_SQL": prediction.strip()
    })

# D. Save to CSV for easy viewing
df = pd.DataFrame(results)
df.to_csv("dpo_results_cot_comparison.csv", index=False)
print("\nDone! Results saved to 'dpo_results_cot_comparison.csv'")

Starting inference for 28 questions...


100%|██████████| 28/28 [05:59<00:00, 12.85s/it]


Done! Results saved to 'dpo_results_cot_comparison.csv'


In [5]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("dpo_results_cot_comparison.csv")

# Loop and print each row separately
for i, row in df.iterrows():
    print(f"{'='*30} ENTRY {i+1} {'='*30}")
    print(f"PROMPT:\n{row['Question']}\n")
    print(f"CHOSEN (GROUND TRUTH):\n{row['Chosen_SQL']}\n")
    print(f"PREDICTED (BY MODEL):\n{row['Predicted_SQL']}\n")
    print(f"{'='*70}\n")

============================== ENTRY 1 ==============================
PROMPT:
System: You are an AI assistant helping a data analyst write SQL queries to answer questions. User: Below, I will provide a DB(Database) schema and a question that can be answered by querying the provided DB.  You will then write out your thought process in detail, followed by a single SQL query enclosed in ‘‘‘sql ...‘‘‘ that answers the question.  Database Schema:  CREATE TABLE frpm ( CDSCode TEXT not null primary key, `Academic Year` TEXT null, `County Code` TEXT null, `District Code` INTEGER null, `School Code` TEXT null, `County Name` TEXT null, `District Name` TEXT null, `School Name` TEXT null, `District Type` TEXT null, `School Type` TEXT null, `Educational Option Type` TEXT null, `NSLP Provision Status` TEXT null, `Charter School (Y/N)` INTEGER null, `Charter School Number` TEXT null, `Charter Funding Type` TEXT null, IRC INTEGER null, `Low Grade` TEXT null, `High Grade` TEXT null, `Enrollment (K-12)`